In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Dropout, Conv2D,  MaxPooling2D, Dense, Flatten, Activation, BatchNormalization
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.regularizers import l2
from tensorflow import config
from skimage.io import imread, imsave
from skimage.transform import resize, rotate
from sklearn.model_selection import train_test_split, StratifiedKFold
import seaborn as sns
import math
from os.path import join
print("GPU is", "available" if config.list_physical_devices('GPU') else "NOT AVAILABLE")

GPU is available


In [2]:
df = pd.read_csv("data/name_images.csv", index_col=0)
df.head()

,fn,label
0,up_left_80-4504980_1981-03-14_2004.jpg,rotated_left
1,upside_down_13-8808513_1960-01-10_1983.jpg,upside_down
2,right_left_84-28509884_1992-08-01_2014.jpg,rotated_left
3,upright_45-20725145_1951-02-08_2007.jpg,upright
4,rotated_left_8-38891708_1957-03-19_2009.jpg,rotated_left


In [3]:
X = df.fn.values
Y = df.label.values
x, x_test, y, y_test = train_test_split(X, Y, test_size=0.04)

In [4]:
class BatchGenerator(Sequence) :
  
        def __init__(self, x_set, y_set, batch_size):
            self.x, self.y = x_set, y_set
            self.batch_size = batch_size

        def __len__(self):
            return math.ceil(len(self.x) / self.batch_size)

        def __getitem__(self, idx):
            batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

            return np.array([imread(join("data","images",file_name))
                   for file_name in batch_x]), np.array(batch_y)

In [5]:
def create_model(learning_rate=0.0001, beta_1=0.80, activation="relu", filters=[32, 32, 64, 64, 512], init_mode='glorot_uniform'):
    model = Sequential()
    model.add(Conv2D(filters[0], (3,3), input_shape=(64,64,3), padding="same"))
    model.add(Activation(activation))
    
    model.add(Conv2D(filters[1], (3, 3), padding="same"))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(filters[2], (3, 3)))
    model.add(Activation(activation))
    model.add(Conv2D(filters[3], (3, 3)))
    model.add(Activation(activation))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(filters[4], kernel_regularizer=l2(0.1)))
    model.add(Activation(activation))
    model.add(Dropout(0.25))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    opt = optimizers.Adam(learning_rate=learning_rate, decay=1e-6, beta_1=beta_1)

    model.compile(loss='categorical_crossentropy', optimizer=opt,
                 metrics=['categorical_accuracy'])
    
    return model

In [6]:
kf = StratifiedKFold(n_splits=10, shuffle=True)
batch_size = 512
loss_list = []
categorical_accuracy_list = []

for train_index, val_index in kf.split(x, y):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = pd.get_dummies(y[train_index]).values, pd.get_dummies(y[val_index]).values
    
    train_batch_generator = BatchGenerator(x_train, y_train, batch_size=batch_size)
    val_batch_generator = BatchGenerator(x_val, y_val, batch_size=batch_size)
    
    model = create_model(beta_1=0.8, filters=[32, 32, 32, 64, 512])
    model.fit(x=train_batch_generator, verbose=1, epochs=10)
    loss, categorical_accuracy = model.evaluate(x=val_batch_generator, verbose=0)
    loss_list.append(loss)
    categorical_accuracy_list.append(categorical_accuracy)

loss_list = np.array(loss_list)
categorical_accuracy_list = np.array(categorical_accuracy_list)

  ...
    to  
  ['...']
Train for 331 steps
Epoch 1/10
331/331 [==============================] - 3036s 9s/step - loss: 49.5159 - categorical_accuracy: 0.7991
Epoch 2/10
331/331 [==============================] - 92s 279ms/step - loss: 14.5050 - categorical_accuracy: 0.9306
Epoch 3/10
331/331 [==============================] - 92s 278ms/step - loss: 5.8169 - categorical_accuracy: 0.9522
Epoch 4/10
331/331 [==============================] - 92s 278ms/step - loss: 2.5226 - categorical_accuracy: 0.9636
Epoch 5/10
331/331 [==============================] - 94s 285ms/step - loss: 1.1513 - categorical_accuracy: 0.9687
Epoch 6/10
331/331 [==============================] - 95s 288ms/step - loss: 0.5669 - categorical_accuracy: 0.9716
Epoch 7/10
331/331 [==============================] - 95s 288ms/step - loss: 0.3107 - categorical_accuracy: 0.9749
Epoch 8/10
331/331 [==============================] - 96s 289ms/step - loss: 0.2037 - categorical_accuracy: 0.9765
Epoch 9/10
331/331 [==============

331/331 [==============================] - 92s 277ms/step - loss: 47.6761 - categorical_accuracy: 0.7881
Epoch 2/10
331/331 [==============================] - 95s 287ms/step - loss: 16.0899 - categorical_accuracy: 0.9263
Epoch 3/10
331/331 [==============================] - 94s 284ms/step - loss: 7.7325 - categorical_accuracy: 0.9518
Epoch 4/10
331/331 [==============================] - 95s 287ms/step - loss: 3.9061 - categorical_accuracy: 0.9628
Epoch 5/10
331/331 [==============================] - 95s 286ms/step - loss: 2.0092 - categorical_accuracy: 0.9678
Epoch 6/10
331/331 [==============================] - 95s 286ms/step - loss: 1.0481 - categorical_accuracy: 0.9718
Epoch 7/10
331/331 [==============================] - 95s 288ms/step - loss: 0.5663 - categorical_accuracy: 0.9738
Epoch 8/10
331/331 [==============================] - 95s 286ms/step - loss: 0.3265 - categorical_accuracy: 0.9757
Epoch 9/10
331/331 [==============================] - 95s 287ms/step - loss: 0.2109 - cat

In [8]:
print(loss_list)
print(categorical_accuracy_list)

[0.12656266 0.20016563 0.16604938 0.29666285 0.58475343 1.41396279
 0.15738043 0.3996809  0.3704887  1.23506475]
[0.9823711  0.97353005 0.9750746  0.9726246  0.97997445 0.9760865
 0.97640604 0.9803473  0.9751278  0.9793886 ]


In [10]:
stats.describe(loss_list)

DescribeResult(nobs=10, minmax=(0.12656266443632744, 1.4139627894839726), mean=0.4950771505566867, variance=0.21211548333243688, skewness=1.2244789972332846, kurtosis=-0.07651518305196792)

In [11]:
stats.describe(categorical_accuracy_list)

DescribeResult(nobs=10, minmax=(0.9726246, 0.9823711), mean=0.97709304, variance=1.0455904e-05, skewness=0.2283092588186264, kurtosis=-1.2185893928592233)

In [12]:
batch_size = 512
train_batch_generator = BatchGenerator(x, pd.get_dummies(y).values, batch_size=batch_size)
test_batch_generator = BatchGenerator(x_test, pd.get_dummies(y_test).values, batch_size=batch_size)
model = create_model(beta_1=0.8, filters=[32, 32, 32, 64, 512])
model.fit(x=train_batch_generator, verbose=0, epochs=10)
model.evaluate(x=test_batch_generator, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
16/16 [==============================] - 123s 8s/step - loss: 0.2261 - categorical_accuracy: 0.9784


[0.22613038029521704, 0.9783998]

In [13]:
def rotate_image(images, orientations):
    for image_name, orientation in zip(images, orientations):
        image = imread(join("data", "images", image_name))
        if orientation == 0:
            image = rotate(image, -90, preserve_range=True).astype(np.uint8)
        elif orientation == 1:
            image = rotate(image, 90, preserve_range=True).astype(np.uint8)
        elif orientation == 3:
            image = rotate(image, 180, preserve_range=True).astype(np.uint8)
            
            
        imsave((join("data","test", image_name)), image)

In [14]:
import warnings
warnings.filterwarnings("ignore")

# Folder "data/test" must be empty to run the cell bellow!

In [15]:
y_test_predict = np.argmax(model.predict(x=test_batch_generator), axis=1)
rotate_image(x_test, y_test_predict)

In [16]:
model.save("model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model\assets
